In [1]:
import pandas as pd
import time
import requests

from bs4 import BeautifulSoup


In [2]:
df = pd.DataFrame({
    'id': [],
    'book_name': [],
    'price': [],
    'url': []
})

In [3]:
def get_books_from_page(soup, data=df):
    """Функция для извлечения книг со страницы категории"""
    
    books = soup.find_all('article', attrs={'class': 'product_pod'})
    
    for book in books:
        book_data = {}
        
        # Название книги
        book_data['book_name'] = book.find('h3').find('a')['title']
        
        # Цена
        book_data['price'] = book.find('p', class_='price_color').text
        
        # Ссылка на книгу
        book_url = book.find('h3').find('a')['href']
        if book_url.startswith('../../../'):
            book_data['url'] = f"https://books.toscrape.com/catalogue/{book_url.replace('../../../', '')}"
        else:
            book_data['url'] = f"https://books.toscrape.com/catalogue/{book_url}"
        
        # ID
        book_data['id'] = len(data) + 1
        
        data.loc[len(data)] = book_data

In [4]:
def get_category_urls():
    """Получает URLs всех категорий"""
    try:
        r = requests.get('https://books.toscrape.com/', timeout=10)
        soup = BeautifulSoup(r.content, 'html.parser')
        
        categories = []
        sidebar = soup.find('div', class_='side_categories')
        if sidebar:
            links = sidebar.find_all('a')
            for link in links[1:]:  # Пропускаем "Books"
                cat_url = link['href']
                full_url = f"https://books.toscrape.com/{cat_url}"
                categories.append(full_url)
        
        return categories
    except Exception as e:
        print(f"Ошибка при получении категорий: {e}")
        return []

In [5]:
# Основной код парсинга
print('Начинаем парсинг книг...')

# Получаем все категории
categories = get_category_urls()
print(f'Найдено категорий: {len(categories)}')

book_count = 0

for category_url in categories:
    if book_count >= 1000:
        break
        
    print(f'Обрабатываем категорию: {category_url}')
    
    page_num = 1
    while True:
        try:
            # Формируем URL страницы категории
            if page_num == 1:
                url = category_url
            else:
                url = category_url.replace('index.html', f'page-{page_num}.html')
            
            r = requests.get(url, timeout=10)
            
            # Проверяем существует ли страница
            if r.status_code == 404:
                break
                
            soup = BeautifulSoup(r.content, 'html.parser')
            
            # Проверяем есть ли книги на странице
            books_on_page = soup.find_all('article', attrs={'class': 'product_pod'})
            if not books_on_page:
                break
            
            # Парсим книги
            get_books_from_page(soup, df)
            book_count = len(df)
            
            print(f'  Страница {page_num}: собрано {len(books_on_page)} книг, всего {book_count}')
            
            # Проверяем лимит
            if book_count >= 1000:
                break
                
            # Проверяем есть ли следующая страница
            next_button = soup.find('li', class_='next')
            if not next_button:
                break
                
            page_num += 1
            time.sleep(0.3)
            
        except Exception as e:
            print(f'Ошибка: {e}')
            break
    
    if book_count >= 1000:
        break

# Обрезаем до 1000 книг
if len(df) > 1000:
    df = df.head(1000)

print(f'\nПарсинг завершен! Собрано книг: {len(df)}')

# Сохраняем в CSV
df.to_csv('books.csv', index=False)
print('Данные сохранены в books.csv')

# Показываем результат
print('\nПервые 5 записей:')
print(df[['id', 'book_name', 'price', 'url']].head())

Начинаем парсинг книг...
Найдено категорий: 50
Обрабатываем категорию: https://books.toscrape.com/catalogue/category/books/travel_2/index.html
  Страница 1: собрано 11 книг, всего 11
Обрабатываем категорию: https://books.toscrape.com/catalogue/category/books/mystery_3/index.html
  Страница 1: собрано 20 книг, всего 31
  Страница 2: собрано 12 книг, всего 43
Обрабатываем категорию: https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html
  Страница 1: собрано 20 книг, всего 63
  Страница 2: собрано 6 книг, всего 69
Обрабатываем категорию: https://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html
  Страница 1: собрано 20 книг, всего 89
  Страница 2: собрано 20 книг, всего 109
  Страница 3: собрано 20 книг, всего 129
  Страница 4: собрано 15 книг, всего 144
Обрабатываем категорию: https://books.toscrape.com/catalogue/category/books/classics_6/index.html
  Страница 1: собрано 19 книг, всего 163
Обрабатываем категорию: https://books.toscrape.